Implementacion del juego Tres en Linea

In [36]:
import numpy as np

class Board():
    #inicializo
    def __init__(self):
        self.state = np.zeros((3,3))

    # valido que movimientos puedo hacer
    def valid_moves(self):
        return [(i, j) for j in range(3) for i in range(3) if self.state[i, j] == 0] # si el valor es 0, puedo usarlo
    
    # actualizo el tablero en funcion de la jugada
    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("Invalid move")
        
    # valido si el juego termino
    def is_game_over(self):
        # comprobar filas y columnas
        if (self.state.sum(axis=0) == 3).any() or (self.state.sum(axis=1) == 3).any():
            return 1 # gano el jugador
        if (self.state.sum(axis=0) == -3).any() or (self.state.sum(axis=1) == -3).any():
            return -1 # gano la maquina
        # comprobar diagonales
        diag_sums = [
            sum([self.state[i, i] for i in range(3)]),
            sum([self.state[i, 3 - i - 1] for i in range(3)]),
        ]
        if diag_sums[0] == 3 or diag_sums[1] == 3:
            return 1
        if diag_sums[0] == -3 or diag_sums[1] == -3:
            return -1
        # comprobar si hay empate
        if self.valid_moves() == []:
            return 0
        return None
    
    # reiniciar el juego
    def reset(self):
        self.state = np.zeros((3,3))

Se encarga de llevar a cabo las partidas del juego Tres en Linea

In [51]:
#Se encarga de llevar a cabo las partidas del juego Tres en Linea
from tqdm import tqdm 

class Game():
    #inicializo
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.board = Board()
        self.players = [player1, player2]
        
    # juegar por su cuenta
    def selfplay(self, rounds=100):
        wins = [0,0] # iniciamos con marcador en 0
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset() # despues de la partida reinicio el tablero
            for player in self.players:
                player.reset() # reinicio el jugador
            game_over = False
            while not game_over: # el juego NO temrino, hago un jugada
                for player in self.players:
                    action = player.move(self.board) # tomo una accion
                    self.board.update(player.symbol, action[0], action[1]) 
                    for player in self.players:
                        player.update(self.board) # guardo el estado
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward() # actualizo el reward / recompenza
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol: # si gano el jugador 1
                    wins[ix] += 1 # sumo 1 al marcador
        return wins
    
    # actualizo el reward
    def reward(self):
        result = self.board.is_game_over()
        if result == 1: # gano el jugador1
            self.players[0].reward(1) # actualizo la reward / recompenza
            self.players[1].reward(0) # actualizo la recompenza / que perdio
        elif result == -1: # gano el jugador2
            self.players[0].reward(0)
            self.players[1].reward(1)
        else: # empate
            self.players[0].reward(0.5) # divido la recompenza
            self.players[1].reward(0.5)

Creo el agente que va a aprender a jugar al Tres en Linea

In [55]:
#Creo el agente que va a aprender a jugar al Tres en Linea
class Agent():
    #inicializo
    def __init__(self, prob_exp=0.5, alpha=0.5, symbol=1):
        self.value_function = {} # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar
        
    # reinicio el jugador
    def reset(self):
        self.positions = []
        
    # actualizo el estado
    def update(self, board):
        self.positions.append(str(board.state.reshape(3*3)))
        
    # tomo una accion
    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        #exploracion
        if explore and np.random.uniform(0,1) < self.prob_exp:
            # tomo una accion aleatoria
            idx = np.random.choice(len(valid_moves)) # elijo una accion al azar dentro de las opciones validas
            return valid_moves[idx]
        
        # vamos a la posicion con mayor valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(3*3))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col
    
    # actualizo la recompenza
    def reward(self, reward):
        # al final de la partida actualizo el valor de cada estado
        for pos in reversed(self.positions):
            if self.value_function.get(pos) is None: # si la posicion no esta en la tabla de posiciones jugadas = 0
                self.value_function[pos] = 0
            self.value_function[pos] += self.alpha * (reward - self.value_function[pos])
            reward = self.value_function[pos]


Creo una instancia de entrenamiento para el agente

In [56]:
#Creo una instancia de entrenamiento para el agente

agent1 = Agent(alpha=0.5)
agent2 = Agent(alpha=0.5)

game = Game(agent1, agent2)
game.selfplay(rounds=10000)

100%|██████████| 10000/10000 [02:07<00:00, 78.54it/s]


[5997, 2315]

Tabla para ver las jugadas del agente

In [60]:
# Tabla con pandas para ver las jugadas del agente
import pandas as pd

fun_value = sorted(agent1.value_function.items(), key=lambda x: x[1], reverse=True)
tabla = pd.DataFrame(fun_value, columns=["Estado", "Valor"])

tabla

,Estado,Valor
0,[ 0. 0. -1. 1. 1. 1. 0. 0. -1.],1.0
1,[ 0. 0. 0. 1. 1. 1. -1. 0. -1.],1.0
2,[ 1. 1. -1. -1. 1. 1. -1. 1. -1.],1.0
3,[-1. 0. -1. 1. 1. 1. 0. 0. 0.],1.0
4,[ 0. 0. -1. 1. 1. 1. -1. 0. 0.],1.0
...,...,...
4663,[ 1. -1. 1. 0. -1. 0. 0. -1. 1.],0.0
4664,[ 1. 0. -1. -1. 0. 0. -1. 1. 1.],0.0
4665,[ 1. 1. -1. 1. -1. 0. -1. -1. 1.],0.0
4666,[ 1. 1. -1. 1. -1. 0. 0. -1. 1.],0.0
